In [34]:
import networkx as nx
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import itertools

In [35]:
data = pd.read_csv('data0.csv')
data["date"] = pd.to_datetime(data["date"]) 
date_begin=pd.to_datetime('2021-01-01')
date_end=pd.to_datetime('2022-12-31')
data=data[(data['date']>=date_begin)&(data['date']<=date_end)]

prediction=pd.read_csv('problem1_prediction.csv')
prediction['ds'] = pd.to_datetime(prediction['ds'])


date_str='2023-01-21'
date_step=1
new_edge_value=1000
load_value=10000
create_value=1000
res_value=100000
step=1
max_edge_flow=505833

data.head()
# print(date+ pd.Timedelta(days=1))

,place1,place2,date,num
0,DC3,DC5,2021-01-01,3
1,DC3,DC10,2021-01-01,4
2,DC3,DC14,2021-01-01,4
3,DC5,DC3,2021-01-01,41
4,DC5,DC9,2021-01-01,3


In [36]:
def cost_func(u,v,not_used,current_flow,max_flow,not_created):
    load_rate=current_flow/max_flow
    res_flow=max_flow-current_flow
    if u=='s' or v=='t':
        return 0
    if '-pre' in u or '-next' in v:
        return 0
    return np.ceil(not_used*new_edge_value +load_rate*load_value+res_value/(res_flow+1)+not_created*create_value)

In [37]:
date=pd.to_datetime(date_str)
delete_points=pd.DataFrame(columns=['place','max_flow'])
delete_points['place']=pd.concat([data['place1'],data['place2']],ignore_index=True).drop_duplicates()
delete_points=delete_points.reset_index()
for index, value in enumerate(delete_points['max_flow'].values):
    in_flow=prediction[(prediction['ds']==date)&(prediction['place1']==delete_points['place'].values[index])]['yhat'].sum()
    out_flow=prediction[(prediction['ds']==date)&(prediction['place2']==delete_points['place'].values[index])]['yhat'].sum()
    delete_points['max_flow'].values[index]=abs(in_flow-out_flow)
delete_points = delete_points.sort_values(by='max_flow', ascending=False)
delete_points.head()
delete_list=delete_points['place'].values.tolist()
print(delete_list)

['DC4', 'DC8', 'DC62', 'DC36', 'DC22', 'DC73', 'DC19', 'DC64', 'DC5', 'DC23', 'DC20', 'DC17', 'DC25', 'DC65', 'DC28', 'DC44', 'DC10', 'DC51', 'DC47', 'DC15', 'DC38', 'DC60', 'DC30', 'DC26', 'DC69', 'DC48', 'DC59', 'DC70', 'DC46', 'DC68', 'DC50', 'DC43', 'DC42', 'DC35', 'DC31', 'DC53', 'DC7', 'DC40', 'DC34', 'DC52', 'DC55', 'DC21', 'DC71', 'DC14', 'DC56', 'DC3', 'DC27', 'DC2', 'DC9', 'DC33', 'DC12', 'DC16', 'DC81', 'DC72', 'DC32', 'DC18', 'DC45', 'DC58', 'DC49', 'DC74', 'DC24', 'DC41', 'DC39', 'DC29', 'DC11', 'DC78', 'DC37', 'DC13', 'DC63', 'DC61', 'DC66', 'DC67', 'DC77', 'DC6', 'DC1', 'DC57', 'DC75', 'DC76', 'DC80', 'DC54', 'DC79']


In [43]:
result_all_list=[]
point_info=pd.DataFrame(columns=['place','res','max_flow','in_flow','out_flow'])
edge_info=pd.DataFrame(columns=['max_flow','place1','place2','current_flow','is_new_flow'])


for delete_num in range(len(delete_list)):
   
    value=str(delete_list[delete_num])

    max_in_sum_1_temp=data[(data['place2']==value)].groupby('date').sum()
    max_in_sum_1=max_in_sum_1_temp['num'].max()
    max_in_sum_2_temp=prediction[(prediction['place2']==value)][['yhat','ds']].groupby('ds').sum()
    max_in_sum_2=max_in_sum_2_temp['yhat'].max()
    if np.isnan(max_in_sum_1):
        max_in_sum_1=0
    if np.isnan(max_in_sum_2):
        max_in_sum_2=0
    max_in_sum=max(max_in_sum_1,max_in_sum_2)
    
    max_out_sum_1_temp=data[(data['place1']==value)].groupby('date').sum()
    max_out_sum_1=max_out_sum_1_temp['num'].max()
    max_out_sum_2_temp=prediction[(prediction['place1']==value)].groupby('ds').sum()
    max_out_sum_2=max_out_sum_2_temp['yhat'].max()
    if np.isnan(max_out_sum_1):
        max_out_sum_1=0
    if np.isnan(max_out_sum_2):
        max_out_sum_2=0
    max_out_sum=max(max_out_sum_1,max_out_sum_2)

    point_info=point_info.append(pd.Series([value,0,max(max_in_sum,max_out_sum),0,0],index=point_info.columns),ignore_index=True)

    point_info.head()
    
    
    unique_routes = data[['place1', 'place2']].drop_duplicates()
    
    for index, row in unique_routes.iterrows():
        place1,place2=row['place1'],row['place2']
        if delete_list[delete_num] in [place1,place2]:
            max_flow_1=data[(data['place1']==place1) & (data['place2']==place2)]['num'].max()
            max_flow_2=prediction[(prediction['place1']==place1) & (prediction['place2']==place2)]['yhat'].max()
            max_flow=max(max_flow_1,max_flow_2)
            # current_flow=prediction[(prediction['ds']==date) & (prediction['place1']==place1) & (prediction['place2']==place2)]['yhat'].values[0]
            edge_info=edge_info.append(pd.Series([max_flow,place1,place2,0,False],index=edge_info.columns),ignore_index=True)

    edge_info.head() 
    
    
    # 创建有向图
    G = nx.DiGraph()
    # 添加节点
    G.add_node('s')  # 源节点
    G.add_node('t')  # 汇节点 

    for index, row in point_info.iterrows():
        point,max_flow,res,in_flow,out_flow=row['place'],row['max_flow'],row['res'],row['in_flow'],row['out_flow']
        G.add_node(point)
        G.add_node(point+'-pre')
        G.add_node(point+'-next')
        
        G.add_edge(point+'-pre', point, capacity=max(max_flow,0), weight=0,flow=0,origin_capacity=max(0,max_flow-in_flow))
        G.add_edge(point, point+'-next', capacity=max(max_flow,0), weight=0,flow=0,origin_capacity=max(0,max_flow-in_flow))

        
       
    for index, row in edge_info.iterrows():
        u,v,max_flow,current_flow,is_new_flow=row['place1'],row['place2'],row['max_flow'],row['current_flow'],row['is_new_flow']
        cost=cost_func(u,v,True,current_flow,max_flow,is_new_flow)
        G.add_edge(u+'-next', v+'-pre', capacity=max(0,max_flow), weight=0,flow=current_flow,origin_capacity=max_flow)

    for index, row in point_info.iterrows():
        place,res,max_flow=row['place'],row['res'],row['max_flow']
        G.add_edge('s', place+'-pre', capacity=1e8, weight=0,flow=0,origin_capacity=1e8)
        G.add_edge(place+'-next', 't', capacity=1e8, weight=0,flow=0,origin_capacity=1e8)

    # flow_to_delete=prediction[(prediction['ds']==date) & (prediction['place2'].isin(delete_list[0:delete_num]))]['yhat'].sum()
    flowDict=nx.max_flow_min_cost(G, 's', 't', capacity='capacity')
    max_flow=sum(flowDict[node1][node2] for node1 in flowDict for node2 in flowDict[node1])
    # if max_flow==0:
    #     break
    print(max_flow)
    result_all_list.append(max_flow)
    print(delete_list[0:delete_num])

    
    

5088456
[]
9243284
['DC4']
12769444
['DC4', 'DC8']
14248172
['DC4', 'DC8', 'DC62']
15856708
['DC4', 'DC8', 'DC62', 'DC36']
17212812
['DC4', 'DC8', 'DC62', 'DC36', 'DC22']
18496224
['DC4', 'DC8', 'DC62', 'DC36', 'DC22', 'DC73']
19667576
['DC4', 'DC8', 'DC62', 'DC36', 'DC22', 'DC73', 'DC19']
21498408
['DC4', 'DC8', 'DC62', 'DC36', 'DC22', 'DC73', 'DC19', 'DC64']
22642736
['DC4', 'DC8', 'DC62', 'DC36', 'DC22', 'DC73', 'DC19', 'DC64', 'DC5']
23540020
['DC4', 'DC8', 'DC62', 'DC36', 'DC22', 'DC73', 'DC19', 'DC64', 'DC5', 'DC23']
25291700
['DC4', 'DC8', 'DC62', 'DC36', 'DC22', 'DC73', 'DC19', 'DC64', 'DC5', 'DC23', 'DC20']
25962512
['DC4', 'DC8', 'DC62', 'DC36', 'DC22', 'DC73', 'DC19', 'DC64', 'DC5', 'DC23', 'DC20', 'DC17']
26579320
['DC4', 'DC8', 'DC62', 'DC36', 'DC22', 'DC73', 'DC19', 'DC64', 'DC5', 'DC23', 'DC20', 'DC17', 'DC25']
27078120
['DC4', 'DC8', 'DC62', 'DC36', 'DC22', 'DC73', 'DC19', 'DC64', 'DC5', 'DC23', 'DC20', 'DC17', 'DC25', 'DC65']
28109960
['DC4', 'DC8', 'DC62', 'DC36', 'DC

In [46]:
result_all_list.reverse()
df=pd.DataFrame(result_all_list,columns=['max_flow'])
df['delete_cnt']=df.reset_index().index
df.to_csv('problem4_all.csv')